In [2]:
import torch
import pandas as pd
import plotly.express as px

df = pd.read_csv('data/dataset.csv')
df= df.set_index(pd.to_datetime(df['date'], format='%Y-%m-%d')).sort_index()

train_df = df["2016":"2022"]
validation_df = df["2022":"2022"]
test_df = df["2023":]

In [4]:
from plotly.subplots import make_subplots
from plotly.graph_objects import Scatter

# Define colors for each type of prediction
colors = {
    "Valeurs réelles": "black",
    "Prévision de SARIMA": "blue",
    "Prévision du TCN": "green",
    "Prévision du LSTM": "orange"
}

global_fig = make_subplots(rows=1, cols=5, subplot_titles=["Agricole", "Résidentiel", "Commercial", "Institutionnel", "Industriel"])

def build_preds_df(sector, mrc="Drummond"):
    sarima_preds = pd.read_csv(f"./arima_predictions/{mrc}_{sector}.csv")
    sarima_preds = sarima_preds.rename(columns={"forecast": "SARIMA"})

    lstm_preds = pd.read_csv(f"./lstm_predictions/LSTM_predictions_vs_true_values_{sector}_{mrc}.csv")
    sarima_preds["LSTM"] = lstm_preds["Predictions"]

    sector_mrc = sector + mrc
    tcn_df = pd.read_csv("tcn_preds2.csv")
    tcn_df = tcn_df.melt(id_vars=["time"], value_vars=tcn_df.columns[1:], var_name="sector_mrc", value_name="output")
    tcn_df["sector_mrc"] = tcn_df["sector_mrc"].str.replace("_", "")
    tcn_p = tcn_df[tcn_df["sector_mrc"] == sector_mrc]
    tcn_p = tcn_p.drop(columns=["sector_mrc"])

    sarima_preds["TCN"] = tcn_p["output"].values

    return sarima_preds

i = 1
for sector in df["sector"].unique():
    show_legend = False if i > 1 else True
    preds_df = build_preds_df(sector)
    global_fig.add_trace(
        Scatter(x=preds_df["date"], y=preds_df["total_kwh"], mode='lines', name="Valeurs réelles", line=dict(width=4, color=colors["Valeurs réelles"], dash='dash'), showlegend=show_legend),
        row=1, col=i)
    global_fig.add_trace(
        Scatter(x=preds_df["date"], y=preds_df["SARIMA"], mode='lines', name="Prévision de SARIMA", line=dict(width=3, color=colors["Prévision de SARIMA"]), showlegend=show_legend),
        row=1, col=i)
    global_fig.add_trace(
        Scatter(x=pd.date_range(start="2023-01-01", end="2023-12-31", periods=10), y=preds_df["TCN"], mode='lines', name="Prévision du TCN", line=dict(width=3, color=colors["Prévision du TCN"]), showlegend=show_legend),
        row=1, col=i)
    global_fig.add_trace(
        Scatter(x=pd.date_range(start="2023-01-01", end="2023-12-31", periods=10), y=preds_df["LSTM"], mode='lines', name="Prévision du LSTM", line=dict(width=3, color=colors["Prévision du LSTM"]), showlegend=show_legend),
        row=1, col=i)

    i += 1
    if i > 5:
        break

global_fig.update_layout(
    # title="Prévisions et consommation pour la MRC de Drummond",
    # title_font=dict(size=28),
    # xaxis_title="Date",
    yaxis_title="Consommation mensuelle (kWh)",
    yaxis=dict(
        title_font=dict(size=20),
        tickfont=dict(size=16),
    ),
    xaxis=dict(
        title_font=dict(size=20),
        tickfont=dict(size=16),
    ),
    legend_title="Légende",
    legend=dict(
        font=dict(
            size=20,
        ),
        yanchor="top",
        y=0.8,
    ),
    margin=dict(l=0, r=0, t=40, b=0),
    width=1500,
    height=500,
    annotations=[
        dict(
            font=dict(size=32),
            showarrow=False,
            text="Agricole",
            x=0.08399999999999999,
            xanchor="center",
            xref="paper",
            y=1.0,
            yanchor="bottom",
            yref="paper",
        ),
        dict(
            font=dict(size=32),
            showarrow=False,
            text="Résidentiel",
            x=0.292,
            xanchor="center",
            xref="paper",
            y=1.0,
            yanchor="bottom",
            yref="paper",
        ),
        dict(
            font=dict(size=32),
            showarrow=False,
            text="Commercial",
            x=0.5,
            xanchor="center",
            xref="paper",
            y=1.0,
            yanchor="bottom",
            yref="paper",
        ),
        dict(
            font=dict(size=32),
            showarrow=False,
            text="Institutionnel",
            x=0.708,
            xanchor="center",
            xref="paper",
            y=1.0,
            yanchor="bottom",
            yref="paper",
        ),
        dict(
            font=dict(size=32),
            showarrow=False,
            text="Industriel",
            x=0.9159999999999999,
            xanchor="center",
            xref="paper",
            y=1.0,
            yanchor="bottom",
            yref="paper",
        ),
    ],
)

# Update tickfont for all subplots
for axis in ['xaxis', 'xaxis2', 'xaxis3', 'xaxis4', 'xaxis5', 'yaxis', 'yaxis2', 'yaxis3', 'yaxis4', 'yaxis5']:
    global_fig.update_layout({axis: dict(tickfont=dict(size=18))})

global_fig.show()
global_fig.write_image("global_fig.png", scale=3)


FileNotFoundError: [Errno 2] No such file or directory: './lstm_predictions/LSTM_predictions_vs_true_values_AGRICOLE_Drummond.csv'

In [20]:
import plotly.express as px

# Plot forecast vs actual
fig = px.line(fore.rename("Prédiction"), title=f"{mrc} - {sector}")
fig.add_trace(go.Scatter(x=y.index, y=y["total_kwh"], mode="lines", name="Valeurs réelles"))
fig.update_layout(legend_title_text="Légende", title=f"Consommation mensuelle pour la MRC de {mrc} - Secteur {sector}", yaxis_title="Valeur (kWh)", xaxis_title="Date")
fig.show()


NameError: name 'fore' is not defined

In [9]:
import pandas as pd 


df = pd.read_csv("results.csv")
df.head()

,MRC,SECTOR,mean_temp__month_rmse,mean_temp__month_mape
0,Abitibi,AGRICOLE,3.031020e+04,0.039937
1,Matawinie,INDUSTRIEL,8.219413e+05,0.170588
2,Matawinie,COMMERCIAL,6.664455e+05,0.041837
3,Matawinie,AGRICOLE,1.788036e+05,0.088196
4,Les Moulins,RÉSIDENTIEL,7.458700e+06,0.042321


In [10]:
# Get mean MAPE for each sector
df.groupby('SECTOR')['mean_temp__month_mape'].mean()

SECTOR
AGRICOLE          0.073025
COMMERCIAL        0.059453
INDUSTRIEL        0.154510
INSTITUTIONNEL    0.058509
RÉSIDENTIEL       0.054587
Name: mean_temp__month_mape, dtype: float64

In [11]:
# Get mean MAPE for each MRC

df.groupby('MRC')['mean_temp__month_mape'].mean().sort_values(ascending=False)

MRC
Lac-Saint-Jean-Est                  0.443822
La Haute-Gaspésie                   0.266998
Mékinac                             0.263477
La Rivière-du-Nord                  0.224387
Les Sources                         0.181747
                                      ...   
Montcalm                            0.034846
Portneuf                            0.034746
Kamouraska                          0.033210
Rimouski-Neigette                   0.032436
Administration régionale Kativik         NaN
Name: mean_temp__month_mape, Length: 88, dtype: float64

In [13]:
import plotly.express as px

px.box(df, x='SECTOR', y='mean_temp__month_mape')